<p> <h1>Continuous AutoCorrelation:</h1>
<h3>Step 1:</h3>Get the distance matrix of the pair for the whole trajectory (729x2084) in this case.
<h3>Step 2:</h3>If the distance value is more than 0.5 make it 1 or else make 0.
<h3>Step 3:</h3>Delete the column of distace whose first value is 0 (means initially it is detached.)
<h3>Step 4:</h3><ol>
    <li>Define an array of nframes/2. First, second and third loops are over time origin, dt and pair.</li>
    <li>For every time origin calculate the denominator(s2) just once. If the time origin itself has valur 0, skip that specific pair.</li>
    <li>If you encounter 0 for a pair, skip the pair from there and keep the pair information(k) in memory.</li>
    <li>If any specific pair is present in the memory skip that pair. If not calculate the numerator(s1).</li>
    <li>If for a specific time origin and dt, all pairs have been scaned, then calculate C(s1/s2) and keep it in an array.</li>
    </ol>
<h3>Step 5:</h3>In the same way calculate for all the time origin and dt.
<h3>Step 6:</h3>Then average over all the time origin.
<h2>Finally you have your Continuous AutoCorrelation function as a function of frame number !!<h/2>
<p>

In [14]:
import numpy as np

In [15]:
data = np.genfromtxt('mindistall.xvg') # It's a file with 729 columns and 2084 rows

In [16]:
data_1_0 = np.where(data[:, 1:] > 0.5, 0, 1)

**<span class="pirk">Hydrogen-Bond Autocorrelation</span>** <br> 
ref: <span class="birk">doi/10.1063.1.4922445</span>, J. Chem. Phys. **142**, 224907 (2015)

The time correlation function used to measure hydrogen bonds in given by <br>
\begin{equation*}
\mathbf{C}_x (t) = \langle \frac{\sum h_{ij} (t_0) h_{ij} (t_0 + t)} {\sum h_{ij} (t_0)^2} \rangle
\end{equation*} <br>
where hij is a binary measure of whether a pairing ij meets the geometric hydrogen bond criteria, hij=1, or not,hij=0. The summation is performed over all the possible pairings, ij (O-N). Angular brackets represent an average over many different starting times in the trajectory. The subscript x referes to the two different definations for measuring hij at future time, continuous or intermittent. <br>
**Continuous:** <br>
Once a particular hydrogen bond is broken, it is then always considered broken even if the bond subsequently reforms. This defination therefore measures the average timne a pair remains intact and yeilds hydrogen bond lifetime. <br>
**Intermittent:** <br>
It allows bonds which were considered broken to be reformed and counted again at a future point in time. It measures the time that a particular hydrogen bonded pair remains in the same vicinity, yeilding information on the structural relaxation time of the polymer.

In [21]:
h = data_1_0

In [22]:
h_1 = h[:, h[0, :]==1]

In [23]:
n_size=int(h_1[:, 0].size/2)

In [ ]:
a = np.empty([n_size, n_size])
#Cx = np.array([])
for i in range(n_size):
    s2 = 0.0
    mem_c = np.array([])
    for j in range(n_size):
        s1 = 0.0
        for k in range(h_1[0, :].size):
            if h_1[i, k] == 0:
                continue
                            
            if j == 0:
                s2 = s2 + h_1[i, k]*h_1[i, k]
                
            if h_1[i+(j+1), k] == 0:
                mem_c = np.append(mem_c, k)
                continue
                
            if k not in mem_c:
                s1 = s1 + h_1[i, k]*h_1[i+(j+1), k]
        C = s1/s2
        a[i, j] = C
        
    #b = np.mean(a)
    #Cx = np.append(Cx, b)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


In [22]:
a

array([[0.99276411, 0.98842258, 0.98552822, ..., 0.59913169, 0.59913169,
        0.59913169],
       [0.99562682, 0.99271137, 0.98979592, ..., 0.60349854, 0.60349854,
        0.60349854],
       [0.99708455, 0.99125364, 0.98833819, ..., 0.60349854, 0.60349854,
        0.60349854],
       ...,
       [0.99387443, 0.98928025, 0.98162328, ..., 0.64471669, 0.64471669,
        0.64471669],
       [0.99389313, 0.98473282, 0.97862595, ..., 0.64274809, 0.64274809,
        0.64274809],
       [0.98931298, 0.98320611, 0.98015267, ..., 0.64274809, 0.64274809,
        0.64274809]])

In [23]:
np.mean(a, axis=0)

array([0.99300649, 0.9883782 , 0.98484002, ..., 0.63949104, 0.63933933,
       0.63918751])

In [24]:
np.savetxt('ac_cc_mean.dat', np.mean(a, axis=0), fmt='%10.5f')